# Bokeh and Python
[1 Tutorial](https://www.youtube.com/watch?v=OJNxE1FjtXU&list=PLlbbWgBRF8EfO4WX13yEWlDUxkHsGPRdV&index=3&t=530s)

[2 bokeh gallery](https://docs.bokeh.org/en/latest/docs/gallery.html)

In [1]:
import pandas as pd
import yfinance as yf

from bokeh.io import curdoc
from bokeh.models import ColumnDataSource, Select, DataTable, TableColumn
from bokeh.layouts import column,row
from bokeh.plotting import figure, show


ModuleNotFoundError: No module named 'bokeh'

In [16]:
DEFAULT_TICKERS = ["AAPL","GOOG","MSFT","NFLX","TSLA"]
START,END = "2021-01-01", "2021-08-28"

In [17]:
def load_ticker(tickers):
    df = yf.download(tickers,start=START,end=END)
    return df["Close"].dropna()

In [18]:
def get_data(t1,t2):
    d = load_ticker(DEFAULT_TICKERS)
    df = d[[t1,t2]]
    returns = df.pct_change().add_suffix("_returns")
    df = pd.concat([df,returns],axis=1)
    df.rename(columns={t1:"t1",t2:"t2",
    t1+"_returns":"t1_returns", t2+"_returns":"t2_returns"},inplace=True)
    return df.dropna()

In [19]:
static_data = load_ticker(DEFAULT_TICKERS)

[*********************100%***********************]  5 of 5 completed


In [20]:
def nix(val,lst):
    '''
    Se elemento è selezionato nella lista uno non duplicarlo nella lista 2 
    '''
    return [x for x in lst if x != val]

ticker1 = Select(value = "AAPL", options = nix("GOOG", DEFAULT_TICKERS))
ticker2 = Select(value = "GOOG", options = nix("AAPL", DEFAULT_TICKERS))

In [21]:
#Source Data
data = get_data(ticker1.value,ticker2.value)
source = ColumnDataSource(data=data)

## Descriptive Stats
stats = round(data.describe().reset_index(),2)
stats_source = ColumnDataSource(data=stats)
stat_columns = [TableColumn(field=col,title=col) for col in stats.columns]
data_table = DataTable(source=stats_source,columns=stat_columns,width=350,height=350,index_position=None)

[*********************100%***********************]  5 of 5 completed


In [22]:
print(data.head())

                    t1           t2  t1_returns  t2_returns
Date                                                       
2021-01-04  129.410004  1728.239990   -0.024719   -0.013494
2021-01-05  131.009995  1740.920044    0.012364    0.007337
2021-01-06  126.599998  1735.290039   -0.033662   -0.003234
2021-01-07  130.919998  1787.250000    0.034123    0.029943
2021-01-08  132.050003  1807.209961    0.008631    0.011168


In [23]:
# Plots
corr_tools = "pan, wheel_zoom, box_select, reset"
tools = "pan, wheel_zoom, xbox_select, reset"

corr = figure(width=350,height=350,tools=corr_tools)
corr.circle("t1_returns","t2_returns",size=2,source=source,
selection_color="firbrick",alpha=0.6,nonselection_alpha=0.1,selection_alpha=0.4 )
#show(corr)

GlyphRenderer(id='1049', ...)

In [24]:
ts1 = figure(width = 700, height = 250, tools = tools,x_axis_type = "datetime",active_drag="xbox_select" )
ts1.line("Date","t1",source=source)
ts1.circle("Date","t1",size=1,source=source,color=None,selection_color="firebrick")

ts2 = figure(width = 700, height = 250, tools = tools,x_axis_type = "datetime",active_drag="xbox_select" )
ts2.x_range=ts1.x_range
ts2.line("Date","t2",source=source)
ts2.circle("Date","t2",size=1,source=source,color=None,selection_color="firebrick")
#show(column(ts1,ts2))


GlyphRenderer(id='1127', ...)

In [25]:
# Callback
def ticker1_change(attrname,old,new):
    ticker2.options = nix(new,DEFAULT_TICKERS)
    update()

def ticker2_change(attrname,old,new):
    ticker1.options = nix(new,DEFAULT_TICKERS)
    update()

def update():
    t1, t2 = ticker1.value,ticker2.value
    df=get_data(t1,t2)
    source.data=df
    stats_source.data = round(df.describe().reset_index(),2)
    corr.title.text = "%s returns vs. returns" % (t1,t2)
    ts1.title.text, ts2.title.text = t1,t2

ticker1.on_change("value", ticker1_change)
ticker2.on_change("value", ticker2_change)

# Layout
widgets = column(ticker1,ticker2,data_table)
main_row = row(corr,widgets)
series = column(ts1,ts2)
layout = column(main_row,series)

show(main_row)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [26]:
# Bokeh Server
curdoc().add_root(layout)
curdoc().title="Stock Data"

RuntimeError: Models must be owned by only a single document, LinearScale(id='1025', ...) is already in a doc